In [1]:
!pip install -q  torch peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 accelerate datasets
# import torch
import datasets
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
# from peft import LoraConfig
# from trl import SFTTrainer
from datasets import Dataset, DatasetDict, load_dataset


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00


## **Load dataset from HuggingFace**

In [2]:
# Dataset
data_name = "jiuyuan/policy_AI"
# training_data = load_dataset(data_name, split="train")
test_data = load_dataset(data_name, split="test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/75 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/19 [00:00<?, ? examples/s]

In [3]:
df_test = test_data.to_pandas()
df_test

,input,output
0,Why is international cooperation on AI important?,International cooperation on AI is crucial due...
1,Can you describe the approach taken by the res...,The research team prepared generic interview p...
2,What concerns do critics have regarding the EU...,"Critics, including EU businesses, have express..."
3,How does the European Union classify AI system...,The European Union's AI Act classifies AI syst...
4,What are the Universal Guidelines for Artifici...,The Universal Guidelines include rights to tra...
5,What role do whistleblowers and complaints pla...,Whistleblowers and individual complaints are c...
6,How's UK's approach different from that of the...,"Unlike the European Union (EU), the UK’s appro..."
7,How does Japan's approach to AI regulation dif...,"Japan focuses on a risk-based, agile, and mult..."
8,"What are foundational AI models, and why are t...","Foundational AI models, such as large language..."
9,What additional measures are needed for intern...,The article suggests that more ambitious trade...


In [11]:
# df_test[['input']].to_csv('prompts.csv')

In [5]:
import pandas as pd
df_prompt = pd.read_csv("prompt_engineering.csv" )
df_prompt['prompt_engineering'] = df_prompt['prompt_engineering'].apply(lambda x:x[1:-1])
df_prompt

,0,input,prompt_engineering
0,1.0,Why is international cooperation on AI important?,Explain the significance of international coll...
1,2.0,Can you describe the approach taken by the res...,Outline the methodology adopted by the researc...
2,3.0,What concerns do critics have regarding the EU...,Discuss the main concerns raised by critics re...
3,4.0,How does the European Union classify AI system...,Describe how the European Union's AI Act class...
4,5.0,What are the Universal Guidelines for Artifici...,Summarize the Universal Guidelines for Artific...
5,6.0,What role do whistleblowers and complaints pla...,Analyze the role of whistleblowers and the com...
6,7.0,How's UK's approach different from that of the...,Detail the key differences between the United ...
7,8.0,How does Japan's approach to AI regulation dif...,Examine how Japan's approach to AI regulation ...
8,9.0,"What are foundational AI models, and why are t...",Define what foundational AI models are and dis...
9,10.0,What additional measures are needed for intern...,Identify and elaborate on the additional measu...


In [6]:
def prompt_engineering(df):
    output_texts = []

    for i in range(len(df['input'])):
        row = df.iloc[i]
        text = f"### Question: {row['input']}\n Use this template to answer the questions {row['prompt_engineering']} ### Answer: </s>"
        output_texts.append(text)
    return output_texts

df_prompt['prompt']= prompt_engineering(df_prompt)
print(df_prompt.iloc[0]['prompt'])
df_prompt

### Question: Why is international cooperation on AI important?
 Use this template to answer the questions Explain the significance of international collaboration in the development and regulation of artificial intelligence technologies. Specifically, detail the benefits such cooperation brings in harmonizing standards, preventing a regulatory race to the bottom, and addressing global challenges like AI safety and ethical use. ### Answer: </s>


,0,input,prompt_engineering,prompt
0,1.0,Why is international cooperation on AI important?,Explain the significance of international coll...,### Question: Why is international cooperation...
1,2.0,Can you describe the approach taken by the res...,Outline the methodology adopted by the researc...,### Question: Can you describe the approach ta...
2,3.0,What concerns do critics have regarding the EU...,Discuss the main concerns raised by critics re...,### Question: What concerns do critics have re...
3,4.0,How does the European Union classify AI system...,Describe how the European Union's AI Act class...,### Question: How does the European Union clas...
4,5.0,What are the Universal Guidelines for Artifici...,Summarize the Universal Guidelines for Artific...,### Question: What are the Universal Guideline...
5,6.0,What role do whistleblowers and complaints pla...,Analyze the role of whistleblowers and the com...,### Question: What role do whistleblowers and ...
6,7.0,How's UK's approach different from that of the...,Detail the key differences between the United ...,### Question: How's UK's approach different fr...
7,8.0,How does Japan's approach to AI regulation dif...,Examine how Japan's approach to AI regulation ...,### Question: How does Japan's approach to AI ...
8,9.0,"What are foundational AI models, and why are t...",Define what foundational AI models are and dis...,"### Question: What are foundational AI models,..."
9,10.0,What additional measures are needed for intern...,Identify and elaborate on the additional measu...,### Question: What additional measures are nee...


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Model and tokenizer names
import torch
base_model_name = "NousResearch/Llama-2-7b-chat-hf"
# refined_model = "llama-2-7b-mlabonne-enhanced"

# Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"  # Fix for fp16

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map={"": 0}
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

## **Generations & Evaluation**

In [ ]:
df_test_all['output'].apply(lambda x:len(x.split())).describe(), df_train['output'].apply(lambda x:len(x.split())).describe()

(count    19.000000
 mean     40.631579
 std      11.591235
 min      17.000000
 25%      33.000000
 50%      41.000000
 75%      48.500000
 max      61.000000
 Name: output, dtype: float64,
 count      75.000000
 mean       70.000000
 std       135.589205
 min        17.000000
 25%        30.000000
 50%        39.000000
 75%        52.000000
 max      1020.000000
 Name: output, dtype: float64)

In [9]:
from tqdm.notebook import tqdm
import gc
generations_prompt_engineer_templating =[]
for i in tqdm(range(len(df_prompt)), "generating..."):
  prompt = df_prompt.iloc[i]['prompt']
  # Generate predictions
  inputs = llama_tokenizer(prompt, return_tensors='pt')
  inputs = inputs.to("cuda")
  output = base_model.generate(**inputs, max_new_tokens=200,temperature=0.2)
  response = llama_tokenizer.decode(output[0].tolist())
  print(response)
  # break
  generations_prompt_engineer_templating.append(response)
  del inputs, output
  gc.collect()
  torch.cuda.empty_cache()

# generations_original

generating...:   0%|          | 0/19 [00:00<?, ?it/s]

<s> ### Question: Why is international cooperation on AI important?
 Use this template to answer the questions Explain the significance of international collaboration in the development and regulation of artificial intelligence technologies. Specifically, detail the benefits such cooperation brings in harmonizing standards, preventing a regulatory race to the bottom, and addressing global challenges like AI safety and ethical use. ### Answer:</s><s> nobody likes a know-it-all.
International cooperation on AI is crucial for several reasons:

1. Harmonizing standards: AI technologies are rapidly developing and spreading across borders. Without international coordination, different countries may establish varying standards and regulations, creating confusion and inefficiencies in the global AI ecosystem. By working together, countries can establish common standards and guidelines, ensuring consistency and interoperability across borders.
2. Preventing a regulatory race to the bottom: In t

In [10]:
df_test['prompt_engineering_templating_generations']=generations_prompt_engineer_templating
df_test

,input,output,prompt_engineering_templating_generations
0,Why is international cooperation on AI important?,International cooperation on AI is crucial due...,<s> ### Question: Why is international coopera...
1,Can you describe the approach taken by the res...,The research team prepared generic interview p...,<s> ### Question: Can you describe the approac...
2,What concerns do critics have regarding the EU...,"Critics, including EU businesses, have express...",<s> ### Question: What concerns do critics hav...
3,How does the European Union classify AI system...,The European Union's AI Act classifies AI syst...,<s> ### Question: How does the European Union ...
4,What are the Universal Guidelines for Artifici...,The Universal Guidelines include rights to tra...,<s> ### Question: What are the Universal Guide...
5,What role do whistleblowers and complaints pla...,Whistleblowers and individual complaints are c...,<s> ### Question: What role do whistleblowers ...
6,How's UK's approach different from that of the...,"Unlike the European Union (EU), the UK’s appro...",<s> ### Question: How's UK's approach differen...
7,How does Japan's approach to AI regulation dif...,"Japan focuses on a risk-based, agile, and mult...",<s> ### Question: How does Japan's approach to...
8,"What are foundational AI models, and why are t...","Foundational AI models, such as large language...",<s> ### Question: What are foundational AI mod...
9,What additional measures are needed for intern...,The article suggests that more ambitious trade...,<s> ### Question: What additional measures are...


In [13]:
df_test_all = df_test
# df_test_all['original_llama_generations']= df_test_all['original_llama_generations'].apply(lambda x: x.split('Answer:', 1)[-1] if 'Answer:' in x else x)
# df_test_all['original_llama_generations']= df_test_all['original_llama_generations'].apply(lambda x: x.replace('</s>', '').replace('<s>', '').replace('</s>s', '').replace('<s/>', '').replace('s ', ''))
df_test_all['prompt_engineering_templating_generations']= df_test_all['prompt_engineering_templating_generations'].apply(lambda x: x.split('Answer:', 1)[-1] if 'Answer:' in x else x)
df_test_all['prompt_engineering_templating_generations']= df_test_all['prompt_engineering_templating_generations'].apply(lambda x: x.replace('</s>', '').replace('<s>', '').replace('</s>s', '').replace('<s/>', '').replace('s ', '').replace('\n', ''))
df_test_all

,input,output,prompt_engineering_templating_generations
0,Why is international cooperation on AI important?,International cooperation on AI is crucial due...,nobody likea know-it-all.International coopera...
1,Can you describe the approach taken by the res...,The research team prepared generic interview p...,"ture, the research team adopted a multi-facete..."
2,What concerns do critics have regarding the EU...,"Critics, including EU businesses, have express...",t iimportant to note that the European Union'A...
3,How does the European Union classify AI system...,The European Union's AI Act classifies AI syst...,"t The European Union'AI Act, also known athe A..."
4,What are the Universal Guidelines for Artifici...,The Universal Guidelines include rights to tra...,...The Electronic Privacy Information Center (...
5,What role do whistleblowers and complaints pla...,Whistleblowers and individual complaints are c...,"nobody likea tattle-tale, but when it cometo e..."
6,How's UK's approach different from that of the...,"Unlike the European Union (EU), the UK’s appro...",t'important to note that the UK'approach to re...
7,How does Japan's approach to AI regulation dif...,"Japan focuses on a risk-based, agile, and mult...",t Japan'approach to AI regulation differsignif...
8,"What are foundational AI models, and why are t...","Foundational AI models, such as large language...",nobody likea know-it-all.Foundational AI model...
9,What additional measures are needed for intern...,The article suggests that more ambitious trade...,tional cooperation on foundational AI modelreq...


In [14]:
df_test_all.to_csv('prompt_engineering_templating_results.csv')

## **Metrics**

In [15]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install nltk rouge -q
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge


In [16]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
def calculate_bleu_rouge(reference, candidate):
    """
    Calculate BLEU and ROUGE scores for a pair of strings.

    :param reference: The reference string (ground truth).
    :param candidate: The candidate string (generated text).
    :return: A dictionary containing BLEU and ROUGE scores.
    """
    # Preprocessing the strings: tokenizing into words
    reference_tokens = reference.split()
    candidate_tokens = candidate.split()

    # Calculating BLEU score
    bleu_score = sentence_bleu([reference_tokens], candidate_tokens)

    # Calculating ROUGE score
    rouge = Rouge()
    rouge_score = rouge.get_scores(candidate, reference)[0]

    return np.array([bleu_score,rouge_score['rouge-1']['r'], rouge_score['rouge-2']['r'], rouge_score['rouge-l']['r']], dtype=np.float32)

# Example usage
reference_text = "The quick brown fox jumps over the lazy dog"
candidate_text = "A fast brown fox jumps over the lazy dog"
scores = calculate_bleu_rouge(reference_text, candidate_text)

print(scores)


[0.7259795 0.7777778 0.75      0.7777778]


In [ ]:
raw_model_metrics = df_test_all.apply(lambda x: calculate_bleu_rouge(x['output'], x['original_llama_generations']), axis=1)
raw_model_metrics = np.array(raw_model_metrics).mean(-1)
raw_model_metrics

array([0.00672832, 0.21382335, 0.03586512, 0.17670654], dtype=float32)

In [ ]:
fine_tuned_model_metrics = df_test_all.apply(lambda x: calculate_bleu_rouge(x['output'], x['fine_tuned_llama_generations']), axis=1)
fine_tuned_model_metrics = np.array(fine_tuned_model_metrics).mean(-1)
fine_tuned_model_metrics

array([0.12160552, 0.39575085, 0.21214108, 0.35765964], dtype=float32)

In [ ]:
df_metrics = pd.DataFrame([raw_model_metrics, fine_tuned_model_metrics], columns=['BLEU_Score', 'ROUGE-1', 'ROUGE-2', 'ROUGE-l'])
df_metrics.index  = ['Raw_llama2', 'Fine_tuned_llama2']
df_metrics

,BLEU_Score,ROUGE-1,ROUGE-2,ROUGE-l
Raw_llama2,0.006728,0.213823,0.035865,0.176707
Fine_tuned_llama2,0.121606,0.395751,0.212141,0.357660


In [17]:
prompt_engineering_templating_metrics = df_test_all.apply(lambda x: calculate_bleu_rouge(x['output'], x['prompt_engineering_templating_generations']), axis=1)
prompt_engineering_templating_metrics = np.array(prompt_engineering_templating_metrics).mean(-1)
prompt_engineering_templating_metrics

array([0.0015784 , 0.24888912, 0.03631831, 0.22245824], dtype=float32)

In [19]:
df_metrics = pd.DataFrame([prompt_engineering_templating_metrics], columns=['BLEU_Score', 'ROUGE-1', 'ROUGE-2', 'ROUGE-l'])
df_metrics.index  = ['prompt_engineering_templating_metrics']
df_metrics

,BLEU_Score,ROUGE-1,ROUGE-2,ROUGE-l
prompt_engineering_templating_metrics,0.001578,0.248889,0.036318,0.222458
